In [95]:
import sqlite3

import pandas as pd

import matplotlib.pyplot as plt

# Import data into pandas dataframe

In [2]:
malaria_deaths_path = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2018/2018-11-13/malaria_deaths.csv'
malaria_deaths_age_path = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2018/2018-11-13/malaria_deaths_age.csv'
malaria_inc_path = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2018/2018-11-13/malaria_inc.csv'

In [25]:
malaria_deaths_df = pd.read_csv(malaria_deaths_path)
malaria_deaths_age_df = pd.read_csv(malaria_deaths_age_path, index_col=0)
malaria_inc_df = pd.read_csv(malaria_inc_path)

print('Original Column Headings')
print(f'malaria_deaths_df columns: {list(malaria_deaths_df.columns)}')
print(f'malaria_deaths_age_df columns: {list(malaria_deaths_age_df.columns)}')
print(f'malaria_inc_df columns: {list(malaria_inc_df.columns)}')

malaria_deaths_df.columns = ['entity', 'code', 'year', 'deaths']
malaria_inc_df.columns = ['entity', 'code', 'year', 'incidence']

print('\nNew Column Headings')
print(f'malaria_deaths_df columns: {list(malaria_deaths_df.columns)}')
print(f'malaria_deaths_age_df columns: {list(malaria_deaths_age_df.columns)}')
print(f'malaria_inc_df columns: {list(malaria_inc_df.columns)}')


Original Column Headings
malaria_deaths_df columns: ['Entity', 'Code', 'Year', 'Deaths - Malaria - Sex: Both - Age: Age-standardized (Rate) (per 100,000 people)']
malaria_deaths_age_df columns: ['entity', 'code', 'year', 'age_group', 'deaths']
malaria_inc_df columns: ['Entity', 'Code', 'Year', 'Incidence of malaria (per 1,000 population at risk) (per 1,000 population at risk)']

New Column Headings
malaria_deaths_df columns: ['entity', 'code', 'year', 'deaths']
malaria_deaths_age_df columns: ['entity', 'code', 'year', 'age_group', 'deaths']
malaria_inc_df columns: ['entity', 'code', 'year', 'incidence']


# Basic Inspection of the dataset

## Inspect head of dataframe

In [75]:
print('malaria_deaths_df')
malaria_deaths_df.head()

malaria_deaths_df


,entity,code,year,deaths
0,Afghanistan,AFG,1990,6.802930
1,Afghanistan,AFG,1991,6.973494
2,Afghanistan,AFG,1992,6.989882
3,Afghanistan,AFG,1993,7.088983
4,Afghanistan,AFG,1994,7.392472


In [76]:
print('malaria_deaths_age_df')
malaria_deaths_age_df.head()

malaria_deaths_age_df


,entity,code,year,age_group,deaths
1,Afghanistan,AFG,1990,Under 5,184.606435
2,Afghanistan,AFG,1991,Under 5,191.658193
3,Afghanistan,AFG,1992,Under 5,197.140197
4,Afghanistan,AFG,1993,Under 5,207.357753
5,Afghanistan,AFG,1994,Under 5,226.209363


In [77]:
print('malaria_inc_df')
malaria_inc_df.head()

malaria_inc_df


,entity,code,year,incidence
0,Afghanistan,AFG,2000,107.100000
1,Afghanistan,AFG,2005,46.500000
2,Afghanistan,AFG,2010,23.900000
3,Afghanistan,AFG,2015,23.600000
4,Algeria,DZA,2000,0.037746


## Identify missing data in the dataset

In [23]:
malaria_deaths_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6156 entries, 0 to 6155
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   entity  6156 non-null   object 
 1   code    5292 non-null   object 
 2   year    6156 non-null   int64  
 3   deaths  6156 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 192.5+ KB


In [26]:
malaria_deaths_age_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30780 entries, 1 to 30780
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   entity     30780 non-null  object 
 1   code       26460 non-null  object 
 2   year       30780 non-null  int64  
 3   age_group  30780 non-null  object 
 4   deaths     30780 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.4+ MB


In [27]:
malaria_inc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   entity     508 non-null    object 
 1   code       400 non-null    object 
 2   year       508 non-null    int64  
 3   incidence  508 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 16.0+ KB


While there were no missing data for most columns, there are missing data for the `code` column for all 3 dataframe.

### Inspect entities that have missing code

In [58]:
def print_entity_with_missing_code(df):
    """
    Print the list of entity that have missing code in the dataframe
    """
    unique_entity = df[['entity', 'code']].drop_duplicates()
    unique_entity_missing_code = unique_entity[unique_entity.code.isnull()]
    print(unique_entity_missing_code)

In [61]:
print('Entity with missing code for malaria_deaths_df')
print_entity_with_missing_code(malaria_deaths_df)

print('\nEntity with missing code for malaria_deaths_age_df')
print_entity_with_missing_code(malaria_deaths_age_df)

print('\nEntity with missing code for malaria_inc_df')
print_entity_with_missing_code(malaria_inc_df)

Entity with missing code for malaria_deaths_df
                            entity code
108           Andean Latin America  NaN
270                    Australasia  NaN
972                      Caribbean  NaN
1026                  Central Asia  NaN
1053                Central Europe  NaN
1080         Central Latin America  NaN
1107    Central Sub-Saharan Africa  NaN
1593                     East Asia  NaN
1620                Eastern Europe  NaN
1647    Eastern Sub-Saharan Africa  NaN
1755                       England  NaN
2349                      High SDI  NaN
2376      High-income Asia Pacific  NaN
2403               High-middle SDI  NaN
2943   Latin America and Caribbean  NaN
3132                       Low SDI  NaN
3159                Low-middle SDI  NaN
3537                    Middle SDI  NaN
3915  North Africa and Middle East  NaN
3942                 North America  NaN
3996              Northern Ireland  NaN
4077                       Oceania  NaN
4644                      Scotlan

`code` are country code of the respective countries. Some entities exist without `code` becuase they are non-country entities, such as geographical regions, cities, economic status. These information might provide insights later during visualisation, thus they are being kept for now.

## Sanity checks on data

In [88]:
malaria_deaths_df.describe()

,year,deaths
count,6156.000000,6156.000000
mean,2003.000000,16.375074
std,7.789514,38.606219
min,1990.000000,0.000000
25%,1996.000000,0.000000
50%,2003.000000,0.020219
75%,2010.000000,3.891035
max,2016.000000,229.228881


In [89]:
malaria_deaths_age_df.describe()

,year,deaths
count,30780.000000,30780.000000
mean,2003.000000,3698.550438
std,7.789007,33539.059413
min,1990.000000,0.000000
25%,1996.000000,0.000000
50%,2003.000000,0.123451
75%,2010.000000,80.505605
max,2016.000000,752025.548675


For `malaria_deaths_df`, `deaths` is the death rate per 100,000 people. In the original dataset, there were not elaborate description about `deaths` for `malaria_deaths_age_df`. The values for `deaths` in `malaria_deaths_age_df` are generally higher than `malaria_deaths_df`. In addition, the maximum value of `deaths` for `malaria_deaths_age_df` is `752,025`. Hence, it is unlikely that `deaths` in `malaria_deaths_age_df` is also death rate per 100,000 people. The values of `deaths` in `malaria_deaths_age_df` are `float` instead of `int`, so it is also unlikely that they are absolute death counts for malaria.

In [90]:
malaria_inc_df.describe()

,year,incidence
count,508.00000,508.000000
mean,2007.50000,141.714565
std,5.59568,176.637900
min,2000.00000,0.000000
25%,2003.75000,6.986310
50%,2007.50000,57.050000
75%,2011.25000,246.050000
max,2015.00000,1741.000000


`incidence` for `malaria_inc_df` suppose to refer to the rate of incidence per 1000 population. Hence, it is not logical for the maximum value for `incidence` to be 1741 (larger than 1000).

**Note:** The time period for `malaria_deaths_df` and `malaria_deaths_age_df` are from 1990 to 2016. However, the time period for `malaria_inc_df` is only from 2000 to 2015. So it will be impossible to do correlation between death and incidence of malaria outside the period 2000 to 2015.

In [99]:
malaria_inc_df[malaria_inc_df['incidence'] > 1000]

,entity,code,year,incidence
464,Turkey,TUR,2000,1741.0


There is only 1 instance of `incidence` (rate of incidence per 1000 population) above 1000, thus it is likely that this is due to an error in the data. This error was made for the `Turkey`.

In [104]:
malaria_inc_df[malaria_inc_df['entity'] == 'Turkey']

,entity,code,year,incidence
464,Turkey,TUR,2000,1000.0
465,Turkey,TUR,2005,295.8
466,Turkey,TUR,2010,0.0
467,Turkey,TUR,2015,0.0


Online articles reported that the number of malaria case in Turkey drop significantly during the period of 2000 to 2015, which was also reflected in the data. 

In [105]:
# Incidence of malaria in turkey in 2000 was corrected to 1000 (the maximum logical value for `incidence`)
malaria_inc_df.iloc[464,3] = 1000
malaria_inc_df[malaria_inc_df['entity'] == 'Turkey']

,entity,code,year,incidence
464,Turkey,TUR,2000,1000.0
465,Turkey,TUR,2005,295.8
466,Turkey,TUR,2010,0.0
467,Turkey,TUR,2015,0.0


In [92]:
# Updating columns title for malaria_deaths_df from `deaths` to `deaths_rate_per_100k`. 
# This is to provide better clarity, and avoid confusion with `deaths` in malaria_deaths_age_df
malaria_deaths_df.columns = ['entity', 'code', 'year', 'deaths_rate_per_100k']

# Export data to SQL database

In [69]:
db_path = 'malaria.db'

with sqlite3.connect(db_path) as sqlite3_con:
    malaria_deaths_df.to_sql('malaria_deaths', con=sqlite3_con, index=False)
    malaria_deaths_age_df.to_sql('malaria_deaths_age_df', con=sqlite3_con, index=False)
    malaria_inc_df.to_sql('malaria_inc_df', con=sqlite3_con, index=False)

# Retrieve data from SQL database

In [83]:
def check_sql(sql):
    if not isinstance(sql, str):
        raise TypeError(f'SQL query need to be in str, but {type(sql)} was provided')
    
    sql = sql.lower()
    if 'select' not in sql:
        raise ValueError('SQL query must at least contain "SELECT"')
    
    if 'from' not in sql:
        raise ValueError('SQL query must at least contain "FROM"')


def sql_to_pandas(sql, db_path='malaria.db'):
    check_sql(sql)
    
    with sqlite3.connect(db_path) as sqlite3_con:
        result_df = pd.read_sql(sql, con=sqlite3_con)
    return result_df

In [86]:
result_df = sql_to_pandas('SELECT * FROM malaria_deaths')